# ChIP seq

In [92]:
import os
import pandas as pd
import sys
import numpy as np
import sys
import re
sys.path.insert(0, '../..')
import itertools

from genepy.utils import helper as h
from genepy.epigenetics import chipseq as chip
from genepy.epigenetics import plot as genepyPlot

import dalmatian as dm
import pyBigWig

from bokeh.plotting import *
from IPython.display import IFrame
import igv
from biomart import BiomartServer
import io

from sklearn.manifold import MDS, TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.cluster import AgglomerativeClustering
from sklearn.mixture import GaussianMixture

output_notebook()
%load_ext autoreload
%autoreload 2

ValueError: expected Resources or a pair of optional Resources, got <class 'bokeh.settings._Unset'>

## MYB_degraded_v1

Chipseq of MYB degron in MV411 cells at 1h of degradation with 0.5$\mu$M VHL (with Drosophilia spike-in control)

Check existing project names. Do __NOT__ overwrite an existing project!

In [3]:
! gsutil ls gs://amlproject/Chip/

gs://amlproject/Chip/GEO/
gs://amlproject/Chip/IRF2BP2_degraded_v1/
gs://amlproject/Chip/IRF2BP2_degraded_v2/
gs://amlproject/Chip/IRF2BP2_degraded_v3/
gs://amlproject/Chip/IRF2BP2_degraded_v4/
gs://amlproject/Chip/IRF2BP2_degraded_v5/
gs://amlproject/Chip/IRF2BP2_degraded_v6/
gs://amlproject/Chip/IRF8_degraded_v1/
gs://amlproject/Chip/MEF2CMEF2D_knockout_v1/
gs://amlproject/Chip/MEF2D_degraded_v1/
gs://amlproject/Chip/MEF2D_degraded_v2/
gs://amlproject/Chip/MEF2D_degraded_v3/
gs://amlproject/Chip/MEF2D_degraded_v4/
gs://amlproject/Chip/PDX/
gs://amlproject/Chip/cobinding/
gs://amlproject/Chip/fastqs/
gs://amlproject/Chip/knockouts_v1/


In [4]:
! gsutil ls gs://transfer-amlproject/

gs://transfer-amlproject/Hello_Mo_Test
gs://transfer-amlproject/201120_MP8439_fastq/
gs://transfer-amlproject/210331_MP8777_fastq/
gs://transfer-amlproject/210514_MP8847_fastq/
gs://transfer-amlproject/210722_MP9119_fastq/
gs://transfer-amlproject/210823_TL9266_fastq/
gs://transfer-amlproject/211012_MP9377_fastq/
gs://transfer-amlproject/211025_MP9377_fastq/
gs://transfer-amlproject/211108_MP9454_fastq/
gs://transfer-amlproject/211217_MP9543_fastq/
gs://transfer-amlproject/220506_MP10011_fastq/
gs://transfer-amlproject/220506_MP10046_fastq/
gs://transfer-amlproject/Cobinding_additional/
gs://transfer-amlproject/IRF2BP2_degraded_v6/
gs://transfer-amlproject/MEF2D_degraded_v4/
gs://transfer-amlproject/MONOMAC_synMEF2CD/
gs://transfer-amlproject/RNP_MEIS1/
gs://transfer-amlproject/RNPv5/
gs://transfer-amlproject/additional_degraded_v1/
gs://transfer-amlproject/chipseq_MEF2C_2h_degraded/
gs://transfer-amlproject/chipseq_MEF2D_degraded/
gs://transfer-amlproject/chipseq_MEFF2CMEF2D_degraded/

In [14]:
project = "MYB_degraded_v1"
loc = "220506_MP10046_fastq" # transfer AML folder
datatype = "chipseq"
project_dir = datatype+"_"+project

In [16]:
project_dir

'chipseq_MYB_degraded_v1'

In [7]:
fastq = ! gsutil ls gs://transfer-amlproject/$loc/*.fastq.gz

for f in fastq:
    print(os.path.basename(f))

20220506_MED1-DMSO_MP10046_S112_R1_001.fastq.gz
20220506_MED1-DMSO_MP10046_S112_R2_001.fastq.gz
20220506_MED1-VHL_MP10046_S116_R1_001.fastq.gz
20220506_MED1-VHL_MP10046_S116_R2_001.fastq.gz
20220506_MYB-DMSO_MP10046_S110_R1_001.fastq.gz
20220506_MYB-DMSO_MP10046_S110_R2_001.fastq.gz
20220506_MYB-VHL_MP10046_S114_R1_001.fastq.gz
20220506_MYB-VHL_MP10046_S114_R2_001.fastq.gz
20220506_MYC-DMSO_MP10046_S111_R1_001.fastq.gz
20220506_MYC-DMSO_MP10046_S111_R2_001.fastq.gz
20220506_MYC-VHL_MP10046_S115_R1_001.fastq.gz
20220506_MYC-VHL_MP10046_S115_R2_001.fastq.gz
20220506_PolII-DMSO_MP10046_S113_R1_001.fastq.gz
20220506_PolII-DMSO_MP10046_S113_R2_001.fastq.gz
20220506_PolII-VHL_MP10046_S117_R1_001.fastq.gz
20220506_PolII-VHL_MP10046_S117_R2_001.fastq.gz


In [8]:
mkdir ../data/$project_dir/ && mkdir ../data/$project_dir/qc/ #make dir in /home/monika/AMLproject/data

In [9]:
! gsutil -m cp gs://transfer-amlproject/$loc/multiqc_report.html ../data/$project_dir/qc/
! gsutil -m cp -r gs://transfer-amlproject/$loc/Reports/ ../data/$project_dir/qc/
! gsutil -m cp -r gs://transfer-amlproject/$loc/multiqc_data/ ../data/$project_dir/qc/

Copying gs://transfer-amlproject/220506_MP10046_fastq/multiqc_report.html...
/ [1/1 files][  1.2 MiB/  1.2 MiB] 100% Done                                    
Operation completed over 1 objects/1.2 MiB.                                      
Copying gs://transfer-amlproject/220506_MP10046_fastq/Reports/20220506_MED1-DMSO_MP10046_S112_R1_001_fastqc.html...
Copying gs://transfer-amlproject/220506_MP10046_fastq/Reports/20220506_MED1-DMSO_MP10046_S112_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/220506_MP10046_fastq/Reports/20220506_MED1-DMSO_MP10046_S112_R2_001_fastqc.html...
Copying gs://transfer-amlproject/220506_MP10046_fastq/Reports/20220506_MED1-DMSO_MP10046_S112_R2_001_fastqc.zip...
Copying gs://transfer-amlproject/220506_MP10046_fastq/Reports/20220506_MED1-VHL_MP10046_S116_R1_001_fastqc.html...
Copying gs://transfer-amlproject/220506_MP10046_fastq/Reports/20220506_MED1-VHL_MP10046_S116_R1_001_fastqc.zip...
Copying gs://transfer-amlproject/220506_MP10046_fastq/Reports/2022050

In [10]:
! gsutil -m cp gs://transfer-amlproject/$loc/*  gs://transfer-amlproject/$project_dir/

Omitting prefix "gs://transfer-amlproject/220506_MP10046_fastq/Reports/". (Did you mean to do cp -r?)
Omitting prefix "gs://transfer-amlproject/220506_MP10046_fastq/multiqc_data/". (Did you mean to do cp -r?)
Copying gs://transfer-amlproject/220506_MP10046_fastq/._multiqc_report.html [Content-Type=text/html]...
Copying gs://transfer-amlproject/220506_MP10046_fastq/20220506_MED1-DMSO_MP10046_S112_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/220506_MP10046_fastq/20220506_MED1-DMSO_MP10046_S112_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/220506_MP10046_fastq/20220506_MED1-VHL_MP10046_S116_R1_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/220506_MP10046_fastq/20220506_MED1-VHL_MP10046_S116_R2_001.fastq.gz [Content-Type=application/octet-stream]...
Copying gs://transfer-amlproject/220506_MP10046_fastq/20220506_MYB-DMSO_MP10046_S110_R1_001.fastq.gz [Content

### download data

In [20]:
! mkdir ../../data/$project_dir && mkdir ../../data/$project_dir/fastqs \
&& gsutil -m cp gs://transfer-amlproject/$project_dir/* ../../data/$project_dir/fastqs

Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/._multiqc_report.html...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MED1-DMSO_MP10046_S112_R1_001.fastq.gz...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MED1-DMSO_MP10046_S112_R2_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MED1-VHL_MP10046_S116_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MED1-VHL_MP10046_S116_R2_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MYB-DMSO_MP10046_S110_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MYB-VHL_MP10046_S114_R1_001.fastq.gz...
Copying gs://trans

In [33]:
! gsutil -m cp gs://transfer-amlproject/$project_dir/* ../../data/$project_dir/fastqs

Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/._multiqc_report.html...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MED1-VHL_MP10046_S116_R2_001.fastq.gz...
==> NOTE: You are downloading one or more large file(s), which would            
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MED1-DMSO_MP10046_S112_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MED1-DMSO_MP10046_S112_R2_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MED1-VHL_MP10046_S116_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MYB-DMSO_MP10046_S110_R1_001.fastq.gz...
Copying gs://transfer-amlproject/chipseq_MYB_degraded_v1/20220506_MYB-DMSO_MP10046_S110_R2_001.fastq.gz...
Copying gs://tran

In [28]:
! rm ../../data/$project_dir/fastqs/multiqc_report.html

In [21]:
fastq_name_list = [i.split("/")[4].split(".")[0] for i in fastq]
len(fastq_name_list)
fastq_name_list

['20220506_MED1-DMSO_MP10046_S112_R1_001',
 '20220506_MED1-DMSO_MP10046_S112_R2_001',
 '20220506_MED1-VHL_MP10046_S116_R1_001',
 '20220506_MED1-VHL_MP10046_S116_R2_001',
 '20220506_MYB-DMSO_MP10046_S110_R1_001',
 '20220506_MYB-DMSO_MP10046_S110_R2_001',
 '20220506_MYB-VHL_MP10046_S114_R1_001',
 '20220506_MYB-VHL_MP10046_S114_R2_001',
 '20220506_MYC-DMSO_MP10046_S111_R1_001',
 '20220506_MYC-DMSO_MP10046_S111_R2_001',
 '20220506_MYC-VHL_MP10046_S115_R1_001',
 '20220506_MYC-VHL_MP10046_S115_R2_001',
 '20220506_PolII-DMSO_MP10046_S113_R1_001',
 '20220506_PolII-DMSO_MP10046_S113_R2_001',
 '20220506_PolII-VHL_MP10046_S117_R1_001',
 '20220506_PolII-VHL_MP10046_S117_R2_001']

In [34]:
rename = {
    "20220506_MYB-DMSO_MP10046_S110": "mp990-MV411_MYB_DMSO-MYB-r1",
    "20220506_MYC-DMSO_MP10046_S111": "mp991-MV411_MYB_DMSO-MYC-r1",
    "20220506_MED1-DMSO_MP10046_S112": "mp992-MV411_MYB_DMSO-MED1-r1",
    "20220506_PolII-DMSO_MP10046_S113": "mp993-MV411_MYB_DMSO-POLII_total-r1",
    
    "20220506_MYB-VHL_MP10046_S114": "mp994-MV411_MYB_VHL-MYB-r1",
    "20220506_MYC-VHL_MP10046_S115": "mp995-MV411_MYB_VHL-MYC-r1",
    "20220506_MED1-VHL_MP10046_S116": "mp996-MV411_MYB_VHL-MED1-r1",
    "20220506_PolII-VHL_MP10046_S117": "mp997-MV411_MYB_VHL-POLII_total-r1",
}

In [35]:
a = ! ls ../../data/$project_dir/fastqs
a

['20220506_MED1-DMSO_MP10046_S112_R1_001.fastq.gz',
 '20220506_MED1-DMSO_MP10046_S112_R2_001.fastq.gz',
 '20220506_MED1-VHL_MP10046_S116_R1_001.fastq.gz',
 '20220506_MED1-VHL_MP10046_S116_R2_001.fastq.gz',
 '20220506_MYB-DMSO_MP10046_S110_R1_001.fastq.gz',
 '20220506_MYB-DMSO_MP10046_S110_R2_001.fastq.gz',
 '20220506_MYB-VHL_MP10046_S114_R1_001.fastq.gz',
 '20220506_MYB-VHL_MP10046_S114_R2_001.fastq.gz',
 '20220506_MYC-DMSO_MP10046_S111_R1_001.fastq.gz',
 '20220506_MYC-DMSO_MP10046_S111_R2_001.fastq.gz',
 '20220506_MYC-VHL_MP10046_S115_R1_001.fastq.gz',
 '20220506_MYC-VHL_MP10046_S115_R2_001.fastq.gz',
 '20220506_PolII-DMSO_MP10046_S113_R1_001.fastq.gz',
 '20220506_PolII-DMSO_MP10046_S113_R2_001.fastq.gz',
 '20220506_PolII-VHL_MP10046_S117_R1_001.fastq.gz',
 '20220506_PolII-VHL_MP10046_S117_R2_001.fastq.gz']

In [36]:
# rename fastqs locally
for val in a:
    rep = val
    for k,v in rename.items():
        rep = rep.replace(k,v)
    ! mv ../../data/$project_dir/fastqs/$val ../../data/$project_dir/fastqs/$rep
    ! gsutil cp gs://transfer-amlproject/$project_dir/$val gs://amlproject/Chip/$project/fastqs/$rep

In [40]:
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret_web.json', '~/.storage.json')
url = "https://docs.google.com/spreadsheets/d/1yFLjYB1McU530JnLgL0QIMAKIkVl3kl0_LCHje2gk8U/edit?ts=5fab1071#gid=738732237"
gsheet = sheets.get(url).sheets[10].to_frame()

In [42]:
gsheet.head()

,id,cell line,replicate,protein,quality,time,paired_end,isNarrow,matching input name,processed,...,total,ratio to droso,unique mapped reads(droso),scaling factor,prefix,Total QC,Peaks,Compensated Wigs,Output,Bigwigs
0,mp100,U937,1,INPUT,NaN,NaN,n,NaN,NaN,Y,...,NaN,NaN,NaN,NaN,https://storage.cloud.google.com/,https://console.cloud.google.com/storage/brows...,https://console.cloud.google.com/storage/brows...,NaN,https://console.cloud.google.com/storage/brows...,https://console.cloud.google.com/storage/brows...
1,mp101,NOMO1,1,INPUT,NaN,NaN,n,NaN,NaN,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mp102,UT7,1,INPUT,NaN,NaN,n,NaN,NaN,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mp106,MV411,1,MYB,NaN,NaN,n,y,mp99,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mp109,M6,1,CEBPA,NaN,NaN,n,y,mp98,Y,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
fastq = ! gsutil ls gs://amlproject/Chip/$project/fastqs/

In [55]:
# design matrix
df = {
"fastq_1": [],
"fastq_2": [],
"antibody": [],
"group": [],
"replicate": [],
"control": []
}
for val in h.grouped(fastq, 2):
    row = gsheet[gsheet.id==val[0].split('/')[-1].split('-')[0]]
    df['group'].append("MV4"+row['name'].values[0].split('-r')[0].split('-MV4')[1])
    df['replicate'].append(row['name'].values[0].split('-r')[1])
    df['fastq_1'].append(val[0])
    df['fastq_2'].append(val[1])
    df['antibody'].append(row['protein'].values[0])
    df['control'].append("INPUT")
df['group'].append('INPUT')
df['replicate'].append(1)
df['fastq_1'].append('gs://amlproject/Chip/fastqs/paired_end/mp845-MV411-INPUT-r2_R1.fastq.gz')
df['fastq_2'].append('gs://amlproject/Chip/fastqs/paired_end/mp845-MV411-INPUT-r2_R2.fastq.gz')
df['antibody'].append("")
df['control'].append("")
df = pd.DataFrame(df)

In [56]:
df

,fastq_1,fastq_2,antibody,group,replicate,control
0,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,MYB,MV411_MYB_DMSO-MYB,1,INPUT
1,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,MYC,MV411_MYB_DMSO-MYC,1,INPUT
2,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,MED1,MV411_MYB_DMSO-MED1,1,INPUT
3,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,POLII_total,MV411_MYB_DMSO-POLII_total,1,INPUT
4,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,MYB,MV411_MYB_VHL-MYB,1,INPUT
5,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,MYC,MV411_MYB_VHL-MYC,1,INPUT
6,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,MED1,MV411_MYB_VHL-MED1,1,INPUT
7,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,POLII_total,MV411_MYB_VHL-POLII_total,1,INPUT
8,gs://amlproject/Chip/fastqs/paired_end/mp845-M...,gs://amlproject/Chip/fastqs/paired_end/mp845-M...,,INPUT,1,


In [50]:
# nextflow CSV
design_file = "../nextflow/"+project+"_design.csv"
design_file_path = os.path.abspath(design_file)
print(design_file_path)
# saved with column order as: group, replicate, fastq_1, fastq_2, antibody, control
df[df.columns[[3,4,0,1,2,5]]].to_csv(design_file_path,index=False)

/home/monika/AMLproject/nextflow/MYB_degraded_v1_design.csv


In [58]:
df = pd.read_csv(design_file) # read in design file
df = df.replace(np.nan, '', regex=True)
df = df[["fastq_1", "fastq_2", "antibody", "group", "replicate", "control"]]
df.iloc[:1,:]

,fastq_1,fastq_2,antibody,group,replicate,control
0,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,gs://amlproject/Chip/MYB_degraded_v1/fastqs/mp...,MYB,MV411_MYB_DMSO-MYB,1,INPUT


### run nextflow

#### about

0. you need to have a google project set up with a billing account
1. you need to activte your APIs this way: https://cloud.google.com/life-sciences/docs/tutorials/nextflow?hl=fr
2. nextflow needs to be installed with this installation command `export NXF_MODE=google && curl https://get.nextflow.io | bash`

In [59]:
! cd /home/monika && sudo ./nextflow log ## to get access to the previous runs

TIMESTAMP          	DURATION  	RUN NAME         	STATUS	REVISION ID	SESSION ID                          	COMMAND                                                                                                                                                                                                                                                                                                                                                                                                          
2021-07-08 20:08:32	-         	hopeful_albattani	-     	fb4924dd0d 	117e88ad-5b5f-4ad3-a368-4ebee2f804c8	nextflow run monikaperez/chipseq --paired_end --seq_center DFCI --email jkobject@gmail.com --input AMLproject/nextflow/MEF2D_degraded_v4_design.csv --genome GRCh38 --max_cpus 16 -profile mpcloud -w 'gs://workamlproject/MEF2Cv4res' --spiking --skip_diff_analysis --outdir 'gs://workamlproject/MEF2Cv4' --tracedir AMLproject/nextflow/MEF2Cdegradedv4/                                        

In [65]:
# write nextflow command to string
export_header = "export NXF_mode=google && sudo"
nextflow_call = "./nextflow run monikaperez/chipseq" # run outside of AMLproject dir
endedness = "--paired_end"
seq_center = "--seq_center " + "'DFCI'"
email = "--email " + "'monikawenperez@gmail.com'"
design_input = "--input " + design_file_path
genome = "--genome " + "GRCh38"
max_cpus = "--max_cpus " + str(16)
profile = "-profile " + "mpcloud"
work_dir = "-w " + "gs://workamlproject/{}res".format(project)
outdir = "--outdir "+"gs://workamlproject/"+project
spiking = "--spiking"
tracedir = "--tracedir AMLproject/nextflow/"+project
#peak_type = "--narrow_peak"

nextflow_args = [export_header, nextflow_call, endedness,
                 seq_center, email, design_input, 
                 genome, max_cpus, profile, 
                 work_dir, outdir, spiking, tracedir,
                 "--skip_diff_analysis"]

# process chips
print("Copy and paste string into command line:\nRun from outside of AMLproject")
print("\n".join(nextflow_args))

Copy and paste string into command line:

export NXF_mode=google && sudo
./nextflow run monikaperez/chipseq
--paired_end
--seq_center 'DFCI'
--email 'monikawenperez@gmail.com'
--input /home/monika/AMLproject/nextflow/MYB_degraded_v1_design.csv
--genome GRCh38
--max_cpus 16
-profile mpcloud
-w gs://workamlproject/MYB_degraded_v1res
--outdir gs://workamlproject/MYB_degraded_v1
--spiking
--tracedir AMLproject/nextflow/MYB_degraded_v1
--skip_diff_analysis


In [66]:
print(" ".join(nextflow_args))

export NXF_mode=google && sudo ./nextflow run monikaperez/chipseq --paired_end --seq_center 'DFCI' --email 'monikawenperez@gmail.com' --input /home/monika/AMLproject/nextflow/MYB_degraded_v1_design.csv --genome GRCh38 --max_cpus 16 -profile mpcloud -w gs://workamlproject/MYB_degraded_v1res --outdir gs://workamlproject/MYB_degraded_v1 --spiking --tracedir AMLproject/nextflow/MYB_degraded_v1 --skip_diff_analysis


move results to main bucket

In [72]:
# move results to main bucket
! gsutil -m cp -r gs://workamlproject/$project/* gs://amlproject/Chip/$project/

Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam.bai [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam [Content-Type=application/octet-st

Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_DMSO-POLII_total_R1.plotFingerprint.qcmetrics.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_DMSO-POLII_total_R1.plotFingerprint.raw.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MED1_R1.plotFingerprint.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MED1_R1.plotFingerprint.qcmetrics.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotFingerprint/MV411_MYB_VHL-MED1_R1.plotFingerprint.raw.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotFingerprint/MV4

Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MED1_R1.plotProfile.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MED1_R1.plotProfile.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MYB_R1.computeMatrix.mat.gz [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MYB_R1.computeMatrix.vals.mat.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MYB_R1.plotHeatmap.mat.tab [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/deepTools/plotProfile/MV411_MYB_VHL-MYB_R1.plotHeatmap.pdf [Content-Type=application/octet-strea

Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_VHL-POLII_total_R1_peaks.gappedPeak [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/MV411_MYB_VHL-POLII_total_R1_peaks.xls [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.annotatePeaks.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.bed [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.boolean.annotatePeaks.txt [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.boolean.intersect.plot.pdf [Content-Type=appli

Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/phantompeakqualtools/MV411_MYB_VHL-POLII_total_R1_spp_correlation_mqc.tsv [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/phantompeakqualtools/MV411_MYB_VHL-POLII_total_R1_spp_nsc_mqc.tsv [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/phantompeakqualtools/MV411_MYB_VHL-POLII_total_R1_spp_rsc_mqc.tsv [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/picard_metrics/INPUT_R1.mLb.clN.CollectMultipleMetrics.alignment_summary_metrics [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/picard_metrics/INPUT_R1.mLb.clN.CollectMultipleMetrics.base_distribution_by_cycle_metrics [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/picard_metrics/INPUT_R1.mLb.clN.Coll

Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/picard_metrics/pdf/MV411_MYB_VHL-MYB_R1.mLb.clN.CollectMultipleMetrics.quality_distribution.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/picard_metrics/pdf/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.base_distribution_by_cycle.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/picard_metrics/pdf/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.insert_size_histogram.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/picard_metrics/pdf/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.quality_by_cycle.pdf [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/picard_metrics/pdf/MV411_MYB_VHL-MYC_R1.mLb.clN.CollectMultipleMetrics.quality_distribution.pdf [Content-Type=application/octet-str

Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/samtools_stats/MV411_MYB_DMSO-POLII_total_R1.mLb.mkD.sorted.bam.flagstat [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/samtools_stats/MV411_MYB_DMSO-POLII_total_R1.mLb.mkD.sorted.bam.idxstats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/samtools_stats/MV411_MYB_DMSO-POLII_total_R1.mLb.mkD.sorted.bam.stats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/samtools_stats/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam.flagstat [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/samtools_stats/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam.idxstats [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/bwa/mergedLibrary/samtools_stats/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam.stats [Cont

Copying gs://workamlproject/MYB_degraded_v1/fastqc/INPUT_R1_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/fastqc/INPUT_R1_T1_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/fastqc/MV411_MYB_DMSO-MED1_R1_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/fastqc/MV411_MYB_DMSO-MED1_R1_T1_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/fastqc/MV411_MYB_DMSO-MYB_R1_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/fastqc/MV411_MYB_DMSO-MYB_R1_T1_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/fastqc/MV411_MYB_DMSO-MYC_R1_T1_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/fastqc/MV411_MYB_DMSO-MYC_R1_T1_2_fastqc.html [Co

Copying gs://workamlproject/MYB_degraded_v1/trim_galore/fastqc/MV411_MYB_VHL-MYC_R1_T1_1_val_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/trim_galore/fastqc/MV411_MYB_VHL-MYC_R1_T1_2_val_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/trim_galore/fastqc/MV411_MYB_VHL-POLII_total_R1_T1_1_val_1_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/trim_galore/fastqc/MV411_MYB_VHL-POLII_total_R1_T1_2_val_2_fastqc.html [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/trim_galore/fastqc/zips/INPUT_R1_T1_1_val_1_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/trim_galore/fastqc/zips/INPUT_R1_T1_2_val_2_fastqc.zip [Content-Type=application/octet-stream]...
Copying gs://workamlproject/MYB_degraded_v1/trim_galore/fastqc/zips/MV411_MYB_DMSO-MED1_R1_T1_1_val_

### get counts for droso scales

In [68]:
# get unique mapped reads(droso)
! gsutil cat gs://amlproject/Chip/$project/droso_aligned/counts/*
! gsutil ls gs://amlproject/Chip/$project/droso_aligned/counts/

10684804
14376964
17145980
17387728
14001076
17053942
17816174
721324
12537200
gs://amlproject/Chip/MYB_degraded_v1/droso_aligned/counts/INPUT_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v1/droso_aligned/counts/MV411_MYB_DMSO-MED1_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v1/droso_aligned/counts/MV411_MYB_DMSO-MYB_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v1/droso_aligned/counts/MV411_MYB_DMSO-MYC_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v1/droso_aligned/counts/MV411_MYB_DMSO-POLII_total_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v1/droso_aligned/counts/MV411_MYB_VHL-MED1_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v1/droso_aligned/counts/MV411_MYB_VHL-MYB_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v1/droso_aligned/counts/MV411_MYB_VHL-MYC_R1_T1_counts.txt
gs://amlproject/Chip/MYB_degraded_v1/droso_aligned/counts/MV411_MYB_VHL-POLII_total_R1_T1_counts.txt


In [69]:
counts = ! gsutil cat gs://amlproject/Chip/$project/droso_aligned/counts/*
samples = ! gsutil ls gs://amlproject/Chip/$project/droso_aligned/counts/

# get unique mapped reads(droso) to sample
for c,s in zip(counts, samples):
    print(s.split("/")[-1]+" : "+c)

INPUT_R1_T1_counts.txt : 10684804
MV411_MYB_DMSO-MED1_R1_T1_counts.txt : 14376964
MV411_MYB_DMSO-MYB_R1_T1_counts.txt : 17145980
MV411_MYB_DMSO-MYC_R1_T1_counts.txt : 17387728
MV411_MYB_DMSO-POLII_total_R1_T1_counts.txt : 14001076
MV411_MYB_VHL-MED1_R1_T1_counts.txt : 17053942
MV411_MYB_VHL-MYB_R1_T1_counts.txt : 17816174
MV411_MYB_VHL-MYC_R1_T1_counts.txt : 721324
MV411_MYB_VHL-POLII_total_R1_T1_counts.txt : 12537200


use the total counts in gs://amlproject/Chip/MYB_degraded_v1/multiqc/broadPeak/multiqc_report.html
<br>use MERGED LIB: SAMTools (filtered) > Percent Mapped data [samtools_alignment_plot-2]
https://console.cloud.google.com/storage/browser/amlproject/Chip/MYB_degraded_v1/multiqc/broadPeak/multiqc_report.html
<br>manually enter total counts in "total" column in AML sample tracker > Chip_Tracker_JK

In [78]:
# copy bams, bigwigs, and macs data to local comp
h.createFoldersFor('../../data/chipseq_'+project+'/bwa/mergedLibrary/')
! gsutil -m cp gs://amlproject/Chip/$project/bwa/mergedLibrary/*.ba* ../../data/chipseq_$project/bwa/mergedLibrary/
! gsutil -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/bigwig/ ../../data/chipseq_$project/bwa/mergedLibrary/
! gsutil -m cp -r gs://amlproject/Chip/$project/bwa/mergedLibrary/macs/ ../../data/chipseq_$project/bwa/mergedLibrary/

Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam.bai...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam.bai...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam...
Copying gs://amlpr

Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MED1/MED1.consensus_peaks.saf...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.annotatePeaks.txt...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.bed...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.boolean.annotatePeaks.txt...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.boolean.intersect.plot.pdf...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.boolean.intersect.txt...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/consensus/MYB/MYB.consensus_peaks.boolean.txt...
Copying gs://amlproject/Chip/MYB_degraded_v1/bwa/mergedLibrary/macs/broadPeak/co

Computing scales from AML sample tracker sheet
* using drosophilia ratio (ratio to droso)
* DMSO: scale = treatment / DMSO
* treatment: scale = DMSO / treatment
* anything above 1 is set to 1 so we are not scaling twice

In [85]:
scales = [ #same as file list order
    1.00, #MV411_MYB_DMSO MED1 R1
    1.00, #MV411_MYB_DMSO MYB R1
    0.06, #MV411_MYB_DMSO MYC R1
    0.81, #MV411_MYB_DMSO POLII_total R1
    
    0.82, #MV411_MYB_VHL MED1 R1
    0.89, #MV411_MYB_VHL MYB R1
    1.00, #MV411_MYB_VHL MYC R1
    1.00  #MV411_MYB_VHL POLII_total R1
]

In [80]:
bams = ! ls ../../data/chipseq_$project/bwa/mergedLibrary/*.bam
bams_short = list(i.split("/")[6] for i in bams)
bams_short

['INPUT_R1.mLb.clN.sorted.bam',
 'MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam',
 'MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam',
 'MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam',
 'MV411_MYB_DMSO-POLII_total_R1.mLb.clN.sorted.bam',
 'MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam',
 'MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam',
 'MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam',
 'MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam']

In [81]:
names = ["MED1_R1", "MYB_R1", "MYC_R1", "POLII_total_R1"]

## on scaled data

In [82]:
! mkdir ../../data/chipseq_$project/diffPeaks/ && ! mkdir ../../data/chipseq_$project/diffData/

In [84]:
wigs = ! ls ../../data/chipseq_$project/bwa/mergedLibrary/bigwig/*.bigWig
wigs

['../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/bigwig/INPUT_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MED1_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MYB_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-MYC_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_DMSO-POLII_total_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MED1_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MYB_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-MYC_R1.bigWig',
 '../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/bigwig/MV411_MYB_VHL-POLII_total_R1.bigWig']

In [89]:
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    print(i)
    bam1 = bams[1+i]
    bam2 = bams[1+len(names)+i]
    print(bams_short[1+i])
    print(bams_short[1+len(names)+i])
    print([scales[i], scales[len(names)+i]])
    print("\n")

0
MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam
[1.0, 0.82]


1
MV411_MYB_DMSO-MYB_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam
[1.0, 0.89]


2
MV411_MYB_DMSO-MYC_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam
[0.06, 1.0]


3
MV411_MYB_DMSO-POLII_total_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam
[0.81, 1.0]




In [90]:
for i in range(int(len(bams)/2)):
    if i < 0:
        continue
    bam1 = bams[1+i]
    bam2 = bams[1+len(names)+i]
    print(bams_short[1+i])
    print(bams_short[1+len(names)+i])
    print(await chip.fullDiffPeak(bam1, bam2, control1 = bams[0], 
                            scaling = [scales[i], scales[len(names)+i]], 
                            directory = "../../data/chipseq_"+project+"/diffData/", 
                            res_directory = "../../data/chipseq_"+project+"/diffPeaks/", 
                            pairedend=True))
    print("\n")

MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam
MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam
doing diff from ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam and ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam
computing the fragment avg size
218
computing the scaling values
b'INFO  @ Mon, 16 May 2022 15:10:07: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 218 -n MV411_MYB_DMSO-MED1_R1 --outdir ../../data/chipseq_MYB_degraded_v1/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_DMSO-MED1_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_DMSO-MED1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.

b'INFO  @ Mon, 16 May 2022 15:32:15: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 218 -n MV411_MYB_VHL-MED1_R1 --outdir ../../data/chipseq_MYB_degraded_v1/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-MED1_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_VHL-MED1_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller dataset.\n# Range for c

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_DMSO-MED1_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_DMSO-MED1_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_VHL-MED1_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_VHL-MED1_R1_control_lambda.bdg --d1 37355451 --d2 42521653 -g 60 -l 218 --o-prefix MV411_MYB_DMSO-MED1_R1_treat_pileup_vs_MV411_MYB_VHL-MED1_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v1/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Mon, 16 May 2022 15:55:52: Read and build treatment 1 bedGraph... \nINFO  @ Mon, 16 May 2022 15:56:43: Read and build control 1 bedGraph... \nINFO  @ Mon, 16 May 2022 15:59:16: Read and build treatment 2 bedGraph... \nINFO  @ Mon, 16 May 2022 16:00:02: Read and build control 2 bedGraph... \nINFO  @ Mon, 16 May 2022 16:09:00: Write peaks... \nINFO  @ Mon, 16 May 2022 16:09:00: 

b'INFO  @ Mon, 16 May 2022 16:34:49: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 221 -n MV411_MYB_VHL-MYB_R1 --outdir ../../data/chipseq_MYB_degraded_v1/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-MYB_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_VHL-MYB_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller dataset.\n# Range for calcu

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_DMSO-MYB_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_DMSO-MYB_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_VHL-MYB_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_VHL-MYB_R1_control_lambda.bdg --d1 41185974 --d2 41360951 -g 60 -l 221 --o-prefix MV411_MYB_DMSO-MYB_R1_treat_pileup_vs_MV411_MYB_VHL-MYB_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v1/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Mon, 16 May 2022 16:56:37: Read and build treatment 1 bedGraph... \nINFO  @ Mon, 16 May 2022 16:57:33: Read and build control 1 bedGraph... \nINFO  @ Mon, 16 May 2022 17:00:05: Read and build treatment 2 bedGraph... \nINFO  @ Mon, 16 May 2022 17:00:53: Read and build control 2 bedGraph... \nINFO  @ Mon, 16 May 2022 17:10:16: Write peaks... \nINFO  @ Mon, 16 May 2022 17:10:16: Done \

b'INFO  @ Mon, 16 May 2022 17:34:27: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 225 -n MV411_MYB_VHL-MYC_R1 --outdir ../../data/chipseq_MYB_degraded_v1/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-MYC_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_VHL-MYC_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards smaller dataset.\n# Range for calcu

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_DMSO-MYC_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_DMSO-MYC_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_VHL-MYC_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_VHL-MYC_R1_control_lambda.bdg --d1 514601933 --d2 26069961 -g 60 -l 225 --o-prefix MV411_MYB_DMSO-MYC_R1_treat_pileup_vs_MV411_MYB_VHL-MYC_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v1/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Mon, 16 May 2022 17:55:07: Read and build treatment 1 bedGraph... \nINFO  @ Mon, 16 May 2022 17:55:50: Read and build control 1 bedGraph... \nINFO  @ Mon, 16 May 2022 17:58:23: Read and build treatment 2 bedGraph... \nINFO  @ Mon, 16 May 2022 17:58:59: Read and build control 2 bedGraph... \nINFO  @ Mon, 16 May 2022 18:07:47: Write peaks... \nINFO  @ Mon, 16 May 2022 18:07:47: Done 

b'INFO  @ Mon, 16 May 2022 18:33:56: \n# Command line: callpeak -B -t ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam -c ../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam --nomodel --extsize 220 -n MV411_MYB_VHL-POLII_total_R1 --outdir ../../data/chipseq_MYB_degraded_v1/diffData/ -f BAMPE\n# ARGUMENTS LIST:\n# name = MV411_MYB_VHL-POLII_total_R1\n# format = BAMPE\n# ChIP-seq file = [\'../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/MV411_MYB_VHL-POLII_total_R1.mLb.clN.sorted.bam\']\n# control file = [\'../../data/chipseq_MYB_degraded_v1/bwa/mergedLibrary/INPUT_R1.mLb.clN.sorted.bam\']\n# effective genome size = 2.70e+09\n# band width = 300\n# model fold = [5, 50]\n# qvalue cutoff = 5.00e-02\n# The maximum gap between significant sites is assigned as the read length/tag size.\n# The minimum length of peaks is assigned as the predicted fragment length "d".\n# Larger dataset will be scaled towards sma

CompletedProcess(args='macs3 bdgdiff --t1 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_DMSO-POLII_total_R1_treat_pileup.bdg --c1 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_DMSO-POLII_total_R1_control_lambda.bdg --t2 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_VHL-POLII_total_R1_treat_pileup.bdg --c2 ../../data/chipseq_MYB_degraded_v1/diffData/MV411_MYB_VHL-POLII_total_R1_control_lambda.bdg --d1 46594586 --d2 42873240 -g 60 -l 220 --o-prefix MV411_MYB_DMSO-POLII_total_R1_treat_pileup_vs_MV411_MYB_VHL-POLII_total_R1_treat_pileup --outdir ../../data/chipseq_MYB_degraded_v1/diffPeaks/', returncode=0, stdout=b'', stderr=b'INFO  @ Mon, 16 May 2022 18:57:37: Read and build treatment 1 bedGraph... \nINFO  @ Mon, 16 May 2022 18:58:28: Read and build control 1 bedGraph... \nINFO  @ Mon, 16 May 2022 19:01:06: Read and build treatment 2 bedGraph... \nINFO  @ Mon, 16 May 2022 19:02:00: Read and build control 2 bedGraph... \nINFO  @ Mon, 16 May 2022 19:11:51: Write peak